# Collect demonstrations

Interactive notebook for collecting demonstrations

In [1]:
%matplotlib auto

import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import json
import random
import socket
from datetime import datetime
from demonstration_recorder import DemonstrationRecorder

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxeng module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController
from robot.al5d_position_controller import PositionController, RobotPosition
from robot.al5d_simulated_position_controller import SimulatedPositionController


from camera.camera_controller import CameraController
from demonstration import Demonstration


Using matplotlib backend: module://matplotlib_inline.backend_inline
Approxeng module not found, cannot use gamepad


In [2]:
experiment = "demonstration_collector"
# picking up a collection specific name
hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    run = "collect_freeform_raven"
elif hostname == "szenes.local":
    run = "collect_no_robot"
else:
    run = "collect_freeform"

exp = Config().get_experiment(experiment, run)
exp_target = Config().get_experiment("demonstration", exp["demonstration_name"])
exp_position_controller = Config().get_experiment(
    exp["exp_position_controller"], 
    exp["run_position_controller"])

exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])
exp_keyboard_controller = Config().get_experiment(exp["exp_keyboard_controller"], exp["run_keyboard_controller"])
exp_gamepad_controller = Config().get_experiment(exp["exp_gamepad_controller"], exp["run_gamepad_controller"])

Hostname is szenes.local
***ExpRun**: Loading pointer config file:
	/Users/lboloni/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/Users/lboloni/Google Drive/My Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-szenes.yaml
***ExpRun**: Configuration for exp/run: demonstration_collector/collect_no_robot successfully loaded
***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/demonstration/_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_cam0_controller

In [3]:
# starting the robot controller
if exp["available_robot"]:
    robot_controller = PositionController(exp_position_controller)
else:
    robot_controller = SimulatedPositionController(exp_position_controller)
# starting the camere controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True


task_dir = exp_target.data_dir()
demoname = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
demo_dir = pathlib.Path(task_dir, demoname)
demo_dir.mkdir(parents=False, exist_ok=False)
print(f"Ok, demonstration with go into\n\t {demo_dir}")

demonstration = Demonstration(exp_target, demoname, parse_old_style = False)
demonstration.metadata["cameras"] = sorted(camera_controller.capture_devs.keys())
demonstration.save_metadata()
# fixme need to add the cameras into the demonstration

Ok, demonstration with go into
	 /Users/lboloni/Documents/Develop/Data/BerryPicker/demonstration/freeform/2025_08_11__12_55_06


OpenCV: out device of bound (0-1): 2
OpenCV: camera failed to properly initialize!


In [4]:
# Performs the recording. The robot is controlled through the primary 
# controller. This is also used to terminate the collection
# The information is collected by the demonstration recorder

controller = exp["primary_controller"]

if controller == "xbox":
    gamepad_controller = GamepadController(exp_gamepad_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration,
        camera_controller=camera_controller, remote_control=gamepad_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    gamepad_controller.demonstration_recorder = demo_recorder
    gamepad_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")
if controller == "keyboard":
    kb_controller = KeyboardController(exp_keyboard_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=kb_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    kb_controller.demonstration_recorder = demo_recorder
    kb_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "program":
    program_controller = ProgramController(
        robot_controller=robot_controller, camera_controller=camera_controller)
    # create wpcount 
    waypoints = []
    wpcount = 10
    while True:
        norm = [0] * 6
        for df in range(6):
            norm[df] = random.random()                
        rp = RobotPosition.from_normalized_vector(exp_position_controller, norm)
        if RobotPosition.limit(exp_position_controller, rp):
            print(f"added waypoint {rp}")
            waypoints.append(rp)
        if len(waypoints) >= wpcount:
            break

    program_controller.waypoints = waypoints
    program_controller.interactive_confirm = False
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=program_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    program_controller.demonstration_recorder = demo_recorder
    demonstration.save_metadata()
    program_controller.control()


INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Position:  height:4.90 distance:5.10 heading:1.50 wrist_angle:-46.50 wrist_rotation:75.50 gripper:95.00
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot done.
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started


added waypoint Position:  height:3.58 distance:9.91 heading:32.74 wrist_angle:-70.06 wrist_rotation:83.60 gripper:26.02
added waypoint Position:  height:1.15 distance:8.92 heading:-35.40 wrist_angle:-4.12 wrist_rotation:61.57 gripper:94.51
added waypoint Position:  height:2.96 distance:8.06 heading:-85.41 wrist_angle:-38.64 wrist_rotation:78.54 gripper:14.31
added waypoint Position:  height:4.17 distance:3.94 heading:-4.93 wrist_angle:-72.91 wrist_rotation:78.28 gripper:52.69
added waypoint Position:  height:3.01 distance:4.58 heading:81.94 wrist_angle:-26.09 wrist_rotation:61.75 gripper:94.73
added waypoint Position:  height:4.66 distance:4.91 heading:30.31 wrist_angle:-54.26 wrist_rotation:75.77 gripper:51.69
added waypoint Position:  height:4.30 distance:9.81 heading:50.45 wrist_angle:-13.21 wrist_rotation:77.26 gripper:65.46
added waypoint Position:  height:3.64 distance:6.28 heading:18.04 wrist_angle:-0.46 wrist_rotation:71.83 gripper:64.31
added waypoint Position:  height:3.15 di

AttributeError: 'SimulatedPositionController' object has no attribute 'angle_controller'

In [ ]:
demonstration.save_metadata()
demonstration.move_to_video(delete_img_files=True)


In [ ]:
print(demonstration.metadata)

In [ ]:
exp